In [ ]:
import os
os.chdir("..")

In [ ]:
import world_bank_data as wb
import pandas as pd
import polars as pl
import missingno as msno
import numpy as np
import pandas as pd
from src.data.data_pull import DataClean
import seaborn as sns
import matplotlib.pyplot as plt

dc = DataClean()

# TODO
- colapse by 5 average
- keep the balance data
- 

In [ ]:
df = dc.pull_wb()
data = pl.read_csv("data/external/countries.csv")
df = df.join(data, on="country", how="inner", validate="m:1")
df = df.select(
    pl.col(
        'year',
        'country',
        'fertility_rate',
        'gdp_per_capita',
        'government_expenditure',
        'trade',
        'life_expectancy',
        'gdp_per_capita_growth',
        'inflation',
        'population_growth',
        'school_primary',
        'gross_capital_formation',
        'domestic_bank',
        'advanced_economy')
)
df = df.filter(pl.col("year") >= 2001)        
df = df.to_pandas()

required_columns = [
        'fertility_rate',
        'gdp_per_capita',
        'government_expenditure',
        'trade',
        'life_expectancy',
        'gdp_per_capita_growth',
        'inflation',
        'population_growth',
        'gross_capital_formation',
        'domestic_bank',
        'advanced_economy']

df['complete'] = df[required_columns].notna().all(axis=1)

# Step 2: Create a pivot table with country as rows and year as columns
pivot = df[df['complete']].pivot_table(index='country', columns='year', values='complete', aggfunc='first')

# Convert boolean to integer (True -> 1, False or NaN -> 0)
pivot = pivot.fillna(0).astype(int)

# Step 3: Brute-force search for the longest consecutive range of years
years = sorted(pivot.columns)
best_combo = {'start': None, 'end': None, 'countries': [], 'n_years': 0}

for i in range(len(years)):
    for j in range(i + 1, len(years) + 1):
        year_range = years[i:j]
        # Select only columns (years) in the current range
        temp = pivot[year_range]
        # Find countries that have full data in all those years
        valid_countries = temp[temp.sum(axis=1) == len(year_range)]
        if len(valid_countries) > 0:
            if len(year_range) > best_combo['n_years'] or (
                len(year_range) == best_combo['n_years'] and len(valid_countries) > len(best_combo['countries'])
            ):
                best_combo = {
                    'start': year_range[0],
                    'end': year_range[-1],
                    'countries': valid_countries.index.tolist(),
                    'n_years': len(year_range)
                }

print(f"Best period: {best_combo['start']} to {best_combo['end']} ({best_combo['n_years']} years)")
print(f"Number of countries: {len(best_combo['countries'])}")
print("Countries:", best_combo['countries'])

In [ ]:
best_combo['countries']

In [ ]:
df[(df["country"].isin(best_combo['countries'])) & (df["year"] <= 2023)].to_csv("data.csv")



In [ ]:
df = df.filter(pl.col("year") >= 1970)
df.write_csv("data.csv")

In [ ]:
df.columns

In [ ]:
# df = df.filter(pl.col("year") >= 2002)
# for cols in  df.columns:
#     df = df.filter(pl.col(f"{cols}").is_not_null())

# units = df.select("country").unique()

# min_year = df["year"].min()
# max_year = df["year"].max()
# years = pl.DataFrame({"year": list(range(min_year, max_year + 1))})

# full_index = units.join(years, how="cross")

# full_df = full_index.join(df, on=["country", "year"], how="left")

# msno.matrix(full_df.to_pandas().sort_values(["country","year"]))

In [ ]:
msno.matrix(df.to_pandas())

In [ ]:
df = df.filter(pl.col("year") >= 1970)
pivoted = df.pivot(
    values="school_primary",
    index="year",
    on="country"
)
msno.bar(pivoted.to_pandas(), sort='descending')

In [ ]:
pivoted = df.pivot(
    values="school_primary",
    index="country",
    on="year"
)
# pivoted.set_index(['year'], inplace=True)
for year2 in range(2005,2022):
    res = pivoted.filter(pl.col(f"{year2}").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
    for year in range(year2 +1,2022):
        tmp = pivoted.filter(pl.col(f"{year}").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
        res = list(set(res) & set(tmp))
    print(f"Amount of countries {len(res)}")
    print(f"start year {year2}")   

In [ ]:
pivoted

In [ ]:
countries = pivoted.filter(pl.col("2005").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
len(countries)

In [ ]:
# countries2 = pivoted.filter(pl.col("2005").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
# countries2 = pivoted.filter(pl.col("2004").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
# res = list(set(countries2) & set(countries2))

for year2 in range(2005,2022):
    res = pivoted.filter(pl.col(f"{year2}").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
    for year in range(year2 +1,2022):
        tmp = pivoted.filter(pl.col(f"{year}").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
        res = list(set(res) & set(tmp))
    print(f"Amount of countries {len(res)}")
    print(f"start year {year2}")   

In [ ]:
df1 = df
res = pivoted.filter(pl.col(1970).is_not_null()).select(pl.col("country")).to_series().unique().to_list()
for year in range(1970,2024):
    tmp = pivoted.filter(pl.col(f"{year}").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
    res = list(set(res) & set(tmp))
# tmp = pivoted.filter(pl.col(f"").is_not_null()).select(pl.col("country")).to_series().unique().to_list()
df1 = df1.filter(pl.col("country").is_in(res))
# df1 = df1.filter((pl.col("year") >= 2002))
df1 = df1.to_pandas().sort_values(["year","country"])
msno.matrix(df1)

In [ ]:
len(df1.select(pl.col("country")).to_series().unique().to_list())

In [ ]:
df.write_csv("data/processed/dataset.csv")